# 1. Read Data
[Github repo](https://github.com/yuejin12345/MachineLearningProjects/tree/master/Azure_Spark)

In [2]:
## Setup our environment by importing required libraries

# Github has been having some timeout issues. This should fix the problem for this dataset.
import socket
socket.setdefaulttimeout(90)

import glob
import os
# Read csv file from URL directly
import pandas as pd

import urllib
from datetime import datetime
# Setup the pyspark environment
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [3]:
# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

Python: 3.7.3 (default, Oct 30 2019, 07:22:34) 
[GCC 5.4.0 20160609]
scipy: 1.2.1
numpy: 1.16.2
matplotlib: 3.0.3
pandas: 0.24.2
sklearn: 0.20.3

In [4]:
# Load libraries
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [5]:
%sql 

SELECT * FROM npi_tax_data_csv;

from_npi,from_tinytax,Addiction,Adolescent,Allergy,Anatomic,Anatomic_Clinical,Anesthesiology,Audiologist,Body_Imaging,Cardiac_Electrophysiology,Cardiology,Cardiothoracic_Vascular,Cardiovascular,Clinical,Counselor,Critical_Care,Cytopathology,Dentist,Dermatology,Dermatopathology,Diagnostic,Diagnostic_Neuroimaging,Diagnostic_Ultrasound,Emergency,Endocrinology,Gastroenterology,General_Practice,Genetics,Geriatric,Hand,Hematology,Hematology_Oncology,Hepatology,Hospitalist,Hyperbaric,Infectious_Disease,Internal,Interventional_Cardiology,Legal_Medicine,Mid_Level,Nephrology,Neurology,Neuromuscular,Neuromusculoskeletal,Neuropathology,Neurophysiology,Neuroradiology,Nuclear,Nuclear_Cardiology,Ob_Gyn,Occupational,Oncology,Ophthalmic,Ophthalmology,Optometrist,Orthopedic,Otolaryngology,Pain,Palliative,Pathology,Pediatric,Pediatrics,Pharmacist,Podiatrist,Primary_Podiatric_Medicine,Psychiatry,Public_Health,Pulmonary,Radiologic_Technologist,Radiology,Rehabilitation,Rehabilitation_Practitioner,Rheumatology,Sleep,Social_Worker,Speech_Language_Pathologist,Sports,Student,Surgery,Transfusion,Transplant,Urology,Vascular_Interventional
1003000126,Internal,0,0,0,0,157,176,0,0,18,0,0,297,0,0,0,0,0,54,11,1645,0,0,714,0,28,436,0,0,0,0,92,0,1172,0,91,1192,31,0,565,398,25,0,31,0,0,219,0,0,0,0,0,0,0,42,0,92,0,0,0,0,0,0,34,0,73,0,111,0,0,27,0,0,0,0,0,0,0,794,0,0,124,136
1003000134,Anatomic_Clinical,0,0,59,0,790,879,41,0,31,0,0,1484,0,0,54,0,0,3728,125,3417,0,0,4294,99,409,490,0,0,0,79,215,0,138,0,69,2822,0,0,632,163,316,0,0,0,0,334,0,0,162,0,121,0,1992,27,0,536,12,0,22,0,0,0,459,12,0,0,144,0,0,66,0,149,0,0,0,0,15,2253,0,0,676,73
1003000142,Anesthesiology,0,0,0,0,81,62,0,0,0,0,0,117,0,0,0,0,0,0,0,692,0,0,164,13,0,136,0,0,0,0,0,0,12,0,0,53,0,0,118,0,19,0,0,0,19,0,0,0,0,0,0,0,0,0,0,0,33,0,0,0,0,0,0,0,0,0,13,0,0,17,0,0,0,0,0,0,0,186,0,0,0,0
1003000209,Mid_Level,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1003000217,Optometrist,0,0,0,0,17,110,0,0,15,0,0,71,0,0,0,0,0,115,0,755,0,0,43,11,20,419,0,0,0,0,0,0,0,0,0,326,115,0,126,13,12,0,0,0,0,0,0,0,0,0,0,0,100,478,0,19,0,0,0,0,0,0,0,0,0,0,22,0,0,0,0,0,0,0,0,0,0,81,0,0,47,0
1003000274,Neurology,0,0,0,0,327,176,0,58,73,47,46,948,0,0,0,0,0,55,11,2082,0,0,778,70,150,640,0,0,0,0,13,0,224,0,46,1084,682,12,308,181,936,0,0,0,0,91,0,0,14,0,12,51,232,13,0,87,56,0,0,0,0,0,18,0,16,0,242,0,0,29,0,32,0,0,0,0,0,335,0,0,174,214
1003000308,Audiologist,0,0,0,0,122,105,0,0,79,57,13,424,0,0,94,0,0,0,0,861,0,0,104,109,129,16,0,0,0,0,34,0,254,0,87,360,29,0,14,96,0,0,0,0,0,0,0,17,0,0,0,0,36,0,0,289,0,0,14,0,0,0,25,0,0,0,105,0,0,0,0,0,0,0,0,0,0,131,0,0,11,67
1003000399,Surgery,0,0,0,0,70,286,18,0,0,0,0,116,0,0,0,0,0,174,0,1191,0,0,109,0,18,466,0,0,0,0,18,0,0,0,14,178,0,0,82,0,81,0,0,0,0,0,0,12,0,0,0,0,129,24,0,13,0,0,0,0,0,0,0,0,0,0,18,0,0,0,0,26,11,0,0,0,12,1469,0,0,61,0
1003000407,General_Practice,0,0,0,0,79,96,0,0,37,0,0,339,0,0,0,0,0,0,0,695,0,13,408,0,29,865,0,0,0,0,75,0,0,0,28,667,213,0,442,26,49,0,0,0,0,0,0,0,0,0,0,0,56,103,0,12,0,0,64,0,0,0,160,0,41,0,239,0,0,0,0,0,0,0,0,0,0,203,0,0,106,54
1003000423,Student,0,0,0,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103,0,0,0,18,0,36,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,73,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 2. Data exploration

We do some preliminary data cleaning and verification, and generate some data exploration figures to help understand the data we will be working with in the remainder of this scenario.

In [7]:
data = spark.table('npi_tax_data_csv') # spark.sql("SELECT * FROM " + npi_tax_data_csv)

In [8]:
dataDF = data.toPandas()

In [9]:
# check missing values in data
dataDF.isnull().sum()

Out[7]: from_npi 0
from_tinytax 0
Addiction 0
Adolescent 0
Allergy 0
Anatomic 0
Anatomic_Clinical 0
Anesthesiology 0
Audiologist 0
Body_Imaging 0
Cardiac_Electrophysiology 0
Cardiology 0
Cardiothoracic_Vascular 0
Cardiovascular 0
Clinical 0
Counselor 0
Critical_Care 0
Cytopathology 0
Dentist 0
Dermatology 0
Dermatopathology 0
Diagnostic 0
Diagnostic_Neuroimaging 0
Diagnostic_Ultrasound 0
Emergency 0
Endocrinology 0
Gastroenterology 0
General_Practice 0
Genetics 0
Geriatric 0
 ..
Ophthalmology 0
Optometrist 0
Orthopedic 0
Otolaryngology 0
Pain 0
Palliative 0
Pathology 0
Pediatric 0
Pediatrics 0
Pharmacist 0
Podiatrist 0
Primary_Podiatric_Medicine 0
Psychiatry 0
Public_Health 0
Pulmonary 0
Radiologic_Technologist 0
Radiology 0
Rehabilitation 0
Rehabilitation_Practitioner 0
Rheumatology 0
Sleep 0
Social_Worker 0
Speech_Language_Pathologist 0
Sports 0
Student 0
Surgery 0
Transfusion 0
Transplant 0
Urology 0
Vascular_Interventional 0
Length: 84, dtype: int64

# 3 Summarize Dataset
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:

Dimensions of the dataset.
Types of the attributes.
Peek at the data itself.
Levels of the class attribute.
Breakdown of the instances in each class.
Statistical summary of all attributes.

## 3.1 Dimensions of Dataset
We can get a quick idea of how many instances (rows) and how many features (columns) the data contains with the dim function.

In [11]:
print (dataDF.shape)

(1183744, 84)

In [12]:
list(dataDF)

Out[13]: ['from_npi',
 'from_tinytax',
 'Addiction',
 'Adolescent',
 'Allergy',
 'Anatomic',
 'Anatomic_Clinical',
 'Anesthesiology',
 'Audiologist',
 'Body_Imaging',
 'Cardiac_Electrophysiology',
 'Cardiology',
 'Cardiothoracic_Vascular',
 'Cardiovascular',
 'Clinical',
 'Counselor',
 'Critical_Care',
 'Cytopathology',
 'Dentist',
 'Dermatology',
 'Dermatopathology',
 'Diagnostic',
 'Diagnostic_Neuroimaging',
 'Diagnostic_Ultrasound',
 'Emergency',
 'Endocrinology',
 'Gastroenterology',
 'General_Practice',
 'Genetics',
 'Geriatric',
 'Hand',
 'Hematology',
 'Hematology_Oncology',
 'Hepatology',
 'Hospitalist',
 'Hyperbaric',
 'Infectious_Disease',
 'Internal',
 'Interventional_Cardiology',
 'Legal_Medicine',
 'Mid_Level',
 'Nephrology',
 'Neurology',
 'Neuromuscular',
 'Neuromusculoskeletal',
 'Neuropathology',
 'Neurophysiology',
 'Neuroradiology',
 'Nuclear',
 'Nuclear_Cardiology',
 'Ob_Gyn',
 'Occupational',
 'Oncology',
 'Ophthalmic',
 'Ophthalmology',
 'Optometrist',
 'Orthopedic',
 'Otolaryngology',
 'Pain',
 'Palliative',
 'Pathology',
 'Pediatric',
 'Pediatrics',
 'Pharmacist',
 'Podiatrist',
 'Primary_Podiatric_Medicine',
 'Psychiatry',
 'Public_Health',
 'Pulmonary',
 'Radiologic_Technologist',
 'Radiology',
 'Rehabilitation',
 'Rehabilitation_Practitioner',
 'Rheumatology',
 'Sleep',
 'Social_Worker',
 'Speech_Language_Pathologist',
 'Sports',
 'Student',
 'Surgery',
 'Transfusion',
 'Transplant',
 'Urology',
 'Vascular_Interventional']

Therea are 1183744 instances and 82 features (X) and 1 predictor (Y).

## 3.2 Types of Attributes
It is a good idea to get an idea of the types of the attributes. They could be doubles, integers, strings, factors and other types.

Knowing the types is important as it will give you an idea of how to better summarize the data you have and the types of transforms you might need to use to prepare the data before you model it. Let’s see the summary of the data.

In [14]:
print (dataDF.dtypes)

from_npi int64
from_tinytax object
Addiction int32
Adolescent int32
Allergy int32
Anatomic int32
Anatomic_Clinical int32
Anesthesiology int32
Audiologist int32
Body_Imaging int32
Cardiac_Electrophysiology int32
Cardiology int32
Cardiothoracic_Vascular int32
Cardiovascular int32
Clinical int32
Counselor int32
Critical_Care int32
Cytopathology int32
Dentist int32
Dermatology int32
Dermatopathology int32
Diagnostic int32
Diagnostic_Neuroimaging int32
Diagnostic_Ultrasound int32
Emergency int32
Endocrinology int32
Gastroenterology int32
General_Practice int32
Genetics int32
Geriatric int32
 ... 
Ophthalmology int32
Optometrist int32
Orthopedic int32
Otolaryngology int32
Pain int32
Palliative int32
Pathology int32
Pediatric int32
Pediatrics int32
Pharmacist int32
Podiatrist int32
Primary_Podiatric_Medicine int32
Psychiatry int32
Public_Health int32
Pulmonary int32
Radiologic_Technologist int32
Radiology int32
Rehabilitation int32
Rehabilitation_Practitioner int32
Rheumatology int32
Sleep int32
Social_Worker int32
Speech_Language_Pathologist int32
Sports int32
Student int32
Surgery int32
Transfusion int32
Transplant int32
Urology int32
Vascular_Interventional int32
Length: 84, dtype: object

## 3.3 Peek at the Data
It is also always a good idea to actually eyeball your data. The first 5 rows of the data are given:

In [16]:
dataDF.head()

,from_npi,from_tinytax,Addiction,Adolescent,Allergy,Anatomic,Anatomic_Clinical,Anesthesiology,Audiologist,Body_Imaging,Cardiac_Electrophysiology,Cardiology,Cardiothoracic_Vascular,Cardiovascular,Clinical,Counselor,Critical_Care,Cytopathology,Dentist,Dermatology,Dermatopathology,Diagnostic,Diagnostic_Neuroimaging,Diagnostic_Ultrasound,Emergency,Endocrinology,Gastroenterology,General_Practice,Genetics,Geriatric,Hand,Hematology,Hematology_Oncology,Hepatology,Hospitalist,Hyperbaric,Infectious_Disease,Internal,Interventional_Cardiology,Legal_Medicine,...,Neuromusculoskeletal,Neuropathology,Neurophysiology,Neuroradiology,Nuclear,Nuclear_Cardiology,Ob_Gyn,Occupational,Oncology,Ophthalmic,Ophthalmology,Optometrist,Orthopedic,Otolaryngology,Pain,Palliative,Pathology,Pediatric,Pediatrics,Pharmacist,Podiatrist,Primary_Podiatric_Medicine,Psychiatry,Public_Health,Pulmonary,Radiologic_Technologist,Radiology,Rehabilitation,Rehabilitation_Practitioner,Rheumatology,Sleep,Social_Worker,Speech_Language_Pathologist,Sports,Student,Surgery,Transfusion,Transplant,Urology,Vascular_Interventional
0,1003000126,Internal,0,0,0,0,157,176,0,0,18,0,0,297,0,0,0,0,0,54,11,1645,0,0,714,0,28,436,0,0,0,0,92,0,1172,0,91,1192,31,0,...,31,0,0,219,0,0,0,0,0,0,0,42,0,92,0,0,0,0,0,0,34,0,73,0,111,0,0,27,0,0,0,0,0,0,0,794,0,0,124,136
1,1003000134,Anatomic_Clinical,0,0,59,0,790,879,41,0,31,0,0,1484,0,0,54,0,0,3728,125,3417,0,0,4294,99,409,490,0,0,0,79,215,0,138,0,69,2822,0,0,...,0,0,0,334,0,0,162,0,121,0,1992,27,0,536,12,0,22,0,0,0,459,12,0,0,144,0,0,66,0,149,0,0,0,0,15,2253,0,0,676,73
2,1003000142,Anesthesiology,0,0,0,0,81,62,0,0,0,0,0,117,0,0,0,0,0,0,0,692,0,0,164,13,0,136,0,0,0,0,0,0,12,0,0,53,0,0,...,0,0,19,0,0,0,0,0,0,0,0,0,0,0,33,0,0,0,0,0,0,0,0,0,13,0,0,17,0,0,0,0,0,0,0,186,0,0,0,0
3,1003000209,Mid_Level,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1003000217,Optometrist,0,0,0,0,17,110,0,0,15,0,0,71,0,0,0,0,0,115,0,755,0,0,43,11,20,419,0,0,0,0,0,0,0,0,0,326,115,0,...,0,0,0,0,0,0,0,0,0,0,100,478,0,19,0,0,0,0,0,0,0,0,0,0,22,0,0,0,0,0,0,0,0,0,0,81,0,0,47,0


## 3.4 Levels of the Doctor Type
The class variable is a factor. A factor is a class that has multiple class labels or levels. Let’s look at the levels:

In [18]:
dataDF.from_tinytax.unique()

Out[16]: array(['Internal', 'Anatomic_Clinical', 'Anesthesiology', 'Mid_Level',
 'Optometrist', 'Neurology', 'Audiologist', 'Surgery',
 'General_Practice', 'Student', 'Urology', 'Cardiovascular',
 'Dermatology', 'Pain', 'Rehabilitation', 'Oncology',
 'Infectious_Disease', 'Ob_Gyn', 'Psychiatry', 'Endocrinology',
 'Diagnostic', 'Emergency', 'Pediatrics',
 'Primary_Podiatric_Medicine', 'Pulmonary', 'Otolaryngology',
 'Ophthalmology', 'Allergy', 'Legal_Medicine',
 'Interventional_Cardiology', 'Gastroenterology', 'Body_Imaging',
 'Hospitalist', 'Nephrology', 'Transplant', 'Orthopedic',
 'Cardiothoracic_Vascular', 'Vascular_Interventional',
 'Critical_Care', 'Hematology_Oncology', 'Neuromusculoskeletal',
 'Hand', 'Social_Worker', 'Palliative', 'Podiatrist', 'Hematology',
 'Cardiac_Electrophysiology', 'Cardiology', 'Diagnostic_Ultrasound',
 'Geriatric', 'Anatomic', 'Pharmacist', 'Rheumatology', 'Sports',
 'Pathology', 'Dermatopathology', 'Radiology', 'Neuroradiology',
 'Nuclear_Cardiology', 'Rehabilitation_Practitioner', 'Addiction',
 'Sleep', 'Speech_Language_Pathologist', 'Counselor', 'Clinical',
 'Adolescent', 'Public_Health', 'Hepatology', 'Hyperbaric',
 'Nuclear', 'Pediatric', 'Neurophysiology', 'Neuropathology',
 'Radiologic_Technologist', 'Occupational', 'Neuromuscular',
 'Diagnostic_Neuroimaging', 'Dentist', 'Transfusion',
 'Cytopathology', 'Genetics', 'Ophthalmic'], dtype=object)

## 3.5 Class Distribution
Let’s now take a look at the number of instances (rows) that belong to each class. We can view this as an absolute count and as a percentage.

In [20]:
# class distribution
dataDF_agg = dataDF.groupby('from_tinytax').size().sort_values(ascending=False)
dataDF_agg

Out[17]: from_tinytax
Mid_Level 402895
General_Practice 116382
Internal 98065
Surgery 81825
Emergency 42313
Optometrist 41287
Anesthesiology 37851
Ob_Gyn 31467
Diagnostic 29402
Psychiatry 22660
Ophthalmology 21984
Cardiovascular 21720
Gastroenterology 15036
Neurology 14745
Dermatology 12689
Hospitalist 12339
Student 11866
Pulmonary 11482
Nephrology 10778
Podiatrist 9930
Urology 9855
Hematology_Oncology 9740
Pain 8988
Anatomic_Clinical 8826
Rehabilitation 8218
Otolaryngology 8176
Audiologist 7813
Oncology 7810
Infectious_Disease 6854
Endocrinology 6731
 ... 
Pathology 645
Nuclear 645
Dermatopathology 643
Body_Imaging 604
Transplant 575
Pharmacist 518
Addiction 515
Geriatric 513
Dentist 433
Pediatric 399
Hyperbaric 392
Counselor 347
Neurophysiology 311
Hepatology 263
Cytopathology 246
Adolescent 242
Radiology 192
Occupational 191
Rehabilitation_Practitioner 187
Public_Health 163
Genetics 151
Diagnostic_Ultrasound 130
Clinical 122
Transfusion 122
Nuclear_Cardiology 105
Neuropathology 93
Diagnostic_Neuroimaging 75
Neuromuscular 71
Ophthalmic 62
Radiologic_Technologist 58
Length: 82, dtype: int64

From the table above, we can see that data is super unbalanced. To have better prediction error, it’s better to make it balanced. One way of balancing the data is bootstrap or we can merge some of the classes into bigger categories. Let's try to keep top 10 professional categories only.

In [22]:
data10 = dataDF.loc[dataDF['from_tinytax'].isin(['from_tinytax', 'Optometrist', 'Anesthesiology', 'Psychiatry', 'Cardiovascular', 'Gastroenterology', 'Neurology', 'Dermatology','Pulmonary','Nephrology','Podiatrist'])]

In [23]:
print (data10.shape)

(198178, 84)

# 4 Split Data into Train and Test Sets
Separating data into training and testing sets is an important part of evaluating data mining models. Typically, when you separate a data set into a training set and testing set, most of the data is used for training, and a smaller portion of the data is used for testing. Analysis Services randomly samples the data to help ensure that the testing and training sets are similar. By using similar data for training and testing, you can minimize the effects of data discrepancies and better understand the characteristics of the model.

We will split the loaded dataset into two, 80% of which we will use to train and validate our models and 20% that we will hold back as a test dataset.

In [25]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data10, test_size=0.2)
print (train.shape)
print (test.shape)

(158542, 84)
(39636, 84)

## 4.1 Statistical Summary
Now finally, we can take a look at a summary of each attribute.

This includes the mean, the min and max values as well as some percentiles (25th, 50th or media and 75th e.g. values at this points if we ordered all the values for an attribute).

In [27]:
# class distribution
train_agg = train.groupby('from_tinytax').size().sort_values(ascending=False)
train_agg

Out[29]: from_tinytax
Optometrist 32950
Anesthesiology 30299
Psychiatry 18138
Cardiovascular 17427
Gastroenterology 12016
Neurology 11878
Dermatology 10215
Pulmonary 9181
Nephrology 8578
Podiatrist 7860
dtype: int64

In [28]:
train.describe()

,from_npi,Addiction,Adolescent,Allergy,Anatomic,Anatomic_Clinical,Anesthesiology,Audiologist,Body_Imaging,Cardiac_Electrophysiology,Cardiology,Cardiothoracic_Vascular,Cardiovascular,Clinical,Counselor,Critical_Care,Cytopathology,Dentist,Dermatology,Dermatopathology,Diagnostic,Diagnostic_Neuroimaging,Diagnostic_Ultrasound,Emergency,Endocrinology,Gastroenterology,General_Practice,Genetics,Geriatric,Hand,Hematology,Hematology_Oncology,Hepatology,Hospitalist,Hyperbaric,Infectious_Disease,Internal,Interventional_Cardiology,Legal_Medicine,Mid_Level,...,Neuromusculoskeletal,Neuropathology,Neurophysiology,Neuroradiology,Nuclear,Nuclear_Cardiology,Ob_Gyn,Occupational,Oncology,Ophthalmic,Ophthalmology,Optometrist,Orthopedic,Otolaryngology,Pain,Palliative,Pathology,Pediatric,Pediatrics,Pharmacist,Podiatrist,Primary_Podiatric_Medicine,Psychiatry,Public_Health,Pulmonary,Radiologic_Technologist,Radiology,Rehabilitation,Rehabilitation_Practitioner,Rheumatology,Sleep,Social_Worker,Speech_Language_Pathologist,Sports,Student,Surgery,Transfusion,Transplant,Urology,Vascular_Interventional
count,1.585420e+05,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,...,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.00000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000,158542.000000
mean,1.499262e+09,0.445428,0.802891,5.976013,14.147431,168.192517,222.185989,4.178722,19.828128,53.498146,5.118486,17.731188,510.392672,1.066834,0.228072,26.789601,2.647866,0.106912,125.206450,19.169312,1143.282581,1.944097,2.421403,326.965076,30.648692,144.403628,417.373850,0.050504,2.623254,0.161427,2.887506,64.311413,1.402240,113.521925,1.552075,59.924380,617.612923,82.431053,3.944002,446.202098,...,3.360283,0.255541,1.453558,18.617975,10.184645,1.925250,17.098889,0.164764,23.378770,0.774255,186.333949,88.434055,0.943939,35.712423,34.105171,6.35366,10.481923,3.112153,5.178325,0.248950,60.031455,5.971049,43.289343,0.137112,148.399238,1.016734,0.992153,39.110457,1.162127,16.771833,4.716826,0.273278,0.222439,0.767973,32.358908,339.426455,1.019250,1.859867,85.793834,40.645873
std,2.879930e+08,7.335521,9.198819,51.440124,125.267481,772.584371,1018.579507,44.584069,152.060255,269.157454,62.535611,95.428313,1849.323229,21.295117,5.128468,153.974133,28.825270,2.628272,657.801325,182.046511,3800.364774,23.767075,25.893275,1262.058295,153.694613,679.067428,1558.951624,1.946060,27.861228,3.579000,30.678637,298.680036,24.201982,486.313317,13.965444,264.649777,2352.220267,480.521236,37.121325,2262.784678,...,38.003612,6.018227,16.102474,114.979293,73.040751,42.174834,126.346081,3.423936,181.134878,13.731569,822.404021,337.065881,13.226438,179.562676,216.406396,38.97578,83.034917,28.362141,50.868750,4.058245,313.216618,45.979235,215.150089,3.630596,562.298056,20.173850,16.617220,223.318297,12.821952,95.336976,36.026969,4.433429,4.206562,8.832349,215.406026,1557.513559,20.619488,23.981558,381.933253,185.193031
min,1.003000e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [29]:
# Correction Matrix Plot
train_cor = train.iloc[:,2:84]
correlations = train_cor.corr()

# plot correlation matrix
plt.rcParams["figure.figsize"] = [16,9]
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,83,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
plt.title('Train Set Correlation Matrix')
plt.show()
display()

In [30]:
train_cor.corr()

,Addiction,Adolescent,Allergy,Anatomic,Anatomic_Clinical,Anesthesiology,Audiologist,Body_Imaging,Cardiac_Electrophysiology,Cardiology,Cardiothoracic_Vascular,Cardiovascular,Clinical,Counselor,Critical_Care,Cytopathology,Dentist,Dermatology,Dermatopathology,Diagnostic,Diagnostic_Neuroimaging,Diagnostic_Ultrasound,Emergency,Endocrinology,Gastroenterology,General_Practice,Genetics,Geriatric,Hand,Hematology,Hematology_Oncology,Hepatology,Hospitalist,Hyperbaric,Infectious_Disease,Internal,Interventional_Cardiology,Legal_Medicine,Mid_Level,Nephrology,...,Neuromusculoskeletal,Neuropathology,Neurophysiology,Neuroradiology,Nuclear,Nuclear_Cardiology,Ob_Gyn,Occupational,Oncology,Ophthalmic,Ophthalmology,Optometrist,Orthopedic,Otolaryngology,Pain,Palliative,Pathology,Pediatric,Pediatrics,Pharmacist,Podiatrist,Primary_Podiatric_Medicine,Psychiatry,Public_Health,Pulmonary,Radiologic_Technologist,Radiology,Rehabilitation,Rehabilitation_Practitioner,Rheumatology,Sleep,Social_Worker,Speech_Language_Pathologist,Sports,Student,Surgery,Transfusion,Transplant,Urology,Vascular_Interventional
Addiction,1.000000,0.083538,0.148960,0.103518,0.174841,0.184582,0.126983,0.161307,0.186514,0.071555,0.167373,0.214353,0.030722,0.111396,0.154149,0.102894,0.064455,0.116181,0.052156,0.225709,0.132766,0.049183,0.230518,0.202695,0.184070,0.206146,0.018589,0.103890,0.079725,0.078357,0.184726,0.041414,0.180236,0.108882,0.221507,0.216803,0.160515,0.048997,0.209364,0.173212,...,0.043857,0.107296,0.086885,0.140247,0.102665,0.040756,0.174278,0.098830,0.178088,0.068967,0.185135,0.126445,0.114525,0.192381,0.153306,0.137603,0.130094,0.058519,0.073478,0.034083,0.162179,0.109467,0.204512,0.083788,0.210813,0.048649,0.032622,0.242371,0.156303,0.165054,0.116805,0.064356,0.052193,0.144210,0.144822,0.204900,0.045850,0.048541,0.196251,0.171708
Adolescent,0.083538,1.000000,0.230701,0.187631,0.274407,0.276205,0.190047,0.131207,0.238379,0.079722,0.234448,0.297538,0.040683,0.022425,0.229670,0.127629,0.123677,0.241827,0.074187,0.322598,0.095471,0.072526,0.315986,0.303849,0.266373,0.294989,0.035061,0.100575,0.134181,0.149729,0.288740,0.035746,0.256807,0.150770,0.278282,0.327129,0.230012,0.144651,0.314785,0.239750,...,0.102941,0.103562,0.090408,0.178754,0.182945,0.056275,0.256671,0.064667,0.249784,0.041577,0.298112,0.206896,0.214388,0.280196,0.250954,0.254088,0.136203,0.120771,0.081838,0.090358,0.229640,0.195762,0.192672,0.092415,0.296204,0.033280,0.050351,0.314597,0.106003,0.255377,0.167205,0.073479,0.073061,0.190392,0.219410,0.318337,0.064819,0.081114,0.305616,0.228396
Allergy,0.148960,0.230701,1.000000,0.519754,0.577499,0.581764,0.554179,0.370552,0.447758,0.168644,0.446596,0.581754,0.102435,0.107016,0.384220,0.372746,0.327605,0.657193,0.321856,0.666165,0.336551,0.121701,0.626538,0.653343,0.557162,0.606931,0.094684,0.203354,0.305055,0.325632,0.612153,0.116146,0.467845,0.305123,0.433247,0.665175,0.424526,0.248403,0.643960,0.400763,...,0.198145,0.209180,0.235838,0.484215,0.343215,0.163430,0.695698,0.248064,0.553584,0.071676,0.740058,0.470668,0.567121,0.735637,0.553766,0.357247,0.292743,0.237130,0.209767,0.157661,0.454148,0.351818,0.426812,0.355933,0.501577,0.075894,0.127769,0.539317,0.159123,0.671857,0.379507,0.134525,0.211342,0.506851,0.461618,0.676299,0.094264,0.196855,0.680574,0.435673
Anatomic,0.103518,0.187631,0.519754,1.000000,0.512961,0.555269,0.380270,0.355026,0.382507,0.144027,0.425275,0.494553,0.142414,0.029395,0.420277,0.317356,0.295821,0.482084,0.202597,0.567613,0.270845,0.114111,0.502335,0.555153,0.511917,0.424640,0.068771,0.176548,0.274159,0.369970,0.514466,0.134818,0.377271,0.249781,0.429638,0.585205,0.324190,0.148585,0.532839,0.371034,...,0.182921,0.269903,0.192931,0.440628,0.398716,0.099299,0.519322,0.196457,0.567427,0.035686,0.589470,0.310740,0.404543,0.582895,0.496389,0.348522,0.236369,0.193552,0.147501,0.088675,0.332370,0.234508,0.353559,0.320252,0.417092,0.085630,0.114623,0.492051,0.176808,0.490470,0.297399,0.085464,0.182138,0.440778,0.427543,0.569068,0.128009,0.260

In [31]:
import seaborn as sns
sns.heatmap(train.corr())
plt.show()
display()

# 5 Visualize Dataset
We now have a basic idea about the data. We need to extend that with some visualizations. We are going to look at two types of plots:

Univariate plots to better understand each attribute. Multivariate plots to better understand the relationships between attributes.

## 5.1 Univariate Plots
We start with some univariate plots, that is, plots of each individual variable.

It is helpful with visualization to have a way to refer to just the input attributes and just the output attributes. Let’s set that up and call the inputs attributes x and the output attribute (or class) y.

In [33]:
# histograms
plt.rcParams["figure.figsize"] = [16,9]
train.hist()
plt.title('Histograms of X Features')
plt.show()
display()

In [34]:
# box and whisker plots
train.plot(kind='box', subplots=True, layout=(9,10), sharex=False, sharey=False)
plt.title('Box Plots of X Features')
plt.show()
display()

In [35]:
plt.rcParams["figure.figsize"] = [4,3]
train['from_tinytax'].value_counts().plot(kind='bar')
plt.title('Histograms of Doctor Types')
plt.show()
display()

## 5.2 Multivariate Plots
Now we can look at the interactions between the variables.

First let’s look at scatterplots of all pairs of attributes and color the points by class. In addition, because the scatterplots show that points for each class are generally separate, we can draw ellipses around them.

In [37]:
from pandas.plotting import scatter_matrix
train_scatter = train.iloc[:,np.r_[1, 2:30]]
scatter_matrix(train_scatter, figsize = (16, 16))
plt.show()
display()